# UniNet 

### UniNet is a SAINT+ like Model. Compared with SAINT+, UniNet puts skill into decoder.
### P.S. Here we also calculate the lag time roughly. (Lag time contributed by https://www.kaggle.com/zlhaaaph)

# Some package import

In [ ]:
# import gc
# import random
# from tqdm import tqdm
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import train_test_split

# import numpy as np 
# import pandas as pd

# import seaborn as sns
# import matplotlib.pyplot as plt

# import torch
# import torch.nn as nn
# import torch.nn.utils.rnn as rnn_utils
# from torch.autograd import Variable
# from torch.utils.data import Dataset, DataLoader
# from pathlib import Path
# import datatable as dt
# import os
# SEED = 9999
# os.environ["PYTHONHASHSEED"] = str(SEED)
# os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
# os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [ ]:
# MAX_SEQ = 180
# ACCEPTED_USER_CONTENT_SIZE = 4
# EMBED_SIZE = 64
# BATCH_SIZE = 32
# DROPOUT = 0.1

# LR = 2e-3
# EPOCHS = 10
# MODEL_PATH = 'sakt.pth'

# Data pre-process

## Load Data

In [ ]:
# path = Path('/kaggle/input')
# assert path.exists()

In [ ]:
# data_types_dict = {
#     'content_type_id': 'bool',
#     'timestamp': 'int64',
#     'user_id': 'int32', 
#     'content_id': 'int16', 
#     'answered_correctly': 'int8', 
#     'prior_question_elapsed_time': 'float32', 
#     'prior_question_had_explanation': 'bool',
#     'task_container_id':'int8',
#     'row_id':'int64'
# }
# target = 'answered_correctly'
# # train_df = dt.fread(path/'riiid-test-answer-prediction/train.csv', columns=set(data_types_dict.keys())).to_pandas()
# train_df = dt.fread('../data/train.csv', columns=set(data_types_dict.keys())).to_pandas()
# # train_df[['prior_question_had_explanation']] = train_df[['prior_question_had_explanation']].shift(-1)

## Pre-process

In [ ]:
# buf = train_df.drop_duplicates(subset=['user_id','task_container_id'],keep='first',inplace=False).sort_values(by='task_container_id')[['user_id','task_container_id','prior_question_elapsed_time']]
# buf['question_elapsed_time'] = buf.groupby('user_id')['prior_question_elapsed_time'].shift(-1)

In [ ]:
# buf = buf.drop(columns=['prior_question_elapsed_time'])

In [ ]:
# train_df = pd.merge(train_df,buf,on=['user_id','task_container_id'],how='left')
# train_df['question_elapsed_time'] = train_df['question_elapsed_time'].fillna(0).astype(int)

In [ ]:
# # 推算lag_time
# train_df['start_time'] = train_df.groupby('user_id')['prior_question_elapsed_time'].shift(-1)
# train_df['start_time'] = train_df['timestamp'] - train_df['start_time']# 开始做题时间
# train_df['lag_time'] = train_df.groupby('user_id')['start_time'].diff()
# train_df['lag_time'] = train_df['lag_time'].fillna(train_df['lag_time'].quantile(0.5))# decoder
# train_df['lag_time'] = (train_df['lag_time']/60000).astype(int)
# train_df.loc[train_df['lag_time']>60*24*7,'lag_time']=60*24*7
# train_df['lag_time'] = train_df['lag_time']-train_df['lag_time'].min()+1# 保证大于0
# train_df['content_id'] = train_df['content_id']+1
# train_df['question_elapsed_time'] = train_df['question_elapsed_time']+1

In [ ]:
# train_df = train_df[train_df.content_type_id == False]
# #arrange by timestamp
# train_df = train_df.sort_values(['timestamp'], ascending=True).reset_index(drop = True)

In [ ]:
# train_df.info()

In [ ]:
# n_content_id = train_df["content_id"].nunique()
# print("number content_id", n_content_id)
# n_elapsed_time = int(train_df["question_elapsed_time"].max())
# print("number elapsed_time", n_elapsed_time)
# n_lag_time = int(train_df["lag_time"].max())
# print("number lag_time", n_lag_time)

# num_content_id_item = n_content_id+1
# num_label_item = 3
# num_skill_item = 2*n_content_id+1
# num_elapsed_time_item = n_elapsed_time+1
# num_lag_time_item = n_lag_time+1

In [ ]:
# group = train_df[['user_id', 'content_id', 'question_elapsed_time','answered_correctly','lag_time']].groupby('user_id').apply(lambda r: (r['content_id'].values, r['question_elapsed_time'].values, r['answered_correctly'].values, r['lag_time'].values))

# Train

## Data Loaders

In [ ]:
# class UniDataset(Dataset):
#     def __init__(self, group, n_skill, max_seq=100):
#         super(UniDataset, self).__init__()
#         self.samples, self.n_skill, self.max_seq = {}, n_skill, max_seq
        
#         self.user_ids = []
#         for i, user_id in enumerate(group.index):
#             if(i % 10000 == 0):
#                 print(f'Processed {i} users')
#             content_id, question_elapsed_time, answered_correctly, lag_time = group[user_id]
#             if len(content_id) >= ACCEPTED_USER_CONTENT_SIZE:
#                 if len(content_id) > self.max_seq:
#                     total_questions = len(content_id)
#                     last_pos = total_questions // self.max_seq
#                     for seq in range(last_pos):
#                         index = f"{user_id}_{seq}"
#                         self.user_ids.append(index)
#                         start = seq * self.max_seq
#                         end = (seq + 1) * self.max_seq
#                         self.samples[index] = (content_id[start:end], question_elapsed_time[start:end], answered_correctly[start:end],lag_time[start:end])
#                     if len(content_id[end:]) >= ACCEPTED_USER_CONTENT_SIZE:
#                         index = f"{user_id}_{last_pos + 1}"
#                         self.user_ids.append(index)
#                         self.samples[index] = (content_id[end:], question_elapsed_time[end:], answered_correctly[end:],lag_time[end:])
#                 else:
#                     index = f'{user_id}'
#                     self.user_ids.append(index)
#                     self.samples[index] = (content_id, question_elapsed_time, answered_correctly,lag_time)
#     def __len__(self):
#         return len(self.user_ids)

#     def __getitem__(self, index):
#         user_id = self.user_ids[index]
#         content_id, question_elapsed_time, answered_correctly, lag_time = self.samples[user_id]
#         seq_len = len(content_id)
        
#         content_id_seq = np.zeros(self.max_seq, dtype=int)
#         question_elapsed_time_seq = np.zeros(self.max_seq, dtype=int)
#         answered_correctly_seq = np.zeros(self.max_seq, dtype=int)
#         lag_time_seq = np.zeros(self.max_seq, dtype=int)
#         label_seq = np.zeros(self.max_seq, dtype=int)
#         if seq_len >= self.max_seq:
#             content_id_seq[:] = content_id[-self.max_seq:]
#             question_elapsed_time_seq[:] = question_elapsed_time[-self.max_seq:]
#             answered_correctly_seq[:] = answered_correctly[-self.max_seq:]+1# 改了
#             label_seq[:] = answered_correctly[-self.max_seq:]
#             lag_time_seq[:] = lag_time[-self.max_seq:]
#         else:
#             content_id_seq[-seq_len:] = content_id
#             question_elapsed_time_seq[-seq_len:] = question_elapsed_time
#             answered_correctly_seq[-seq_len:] = answered_correctly+1# 改了
#             label_seq[-seq_len:] = answered_correctly
#             lag_time_seq[-seq_len:] = lag_time
            
#         content_seq = content_id_seq[1:]
#         elapsed_time_seq = question_elapsed_time_seq[1:]
#         label_seq = label_seq[1:]
#         lag_time_seq = lag_time_seq[1:]
        
#         skill_seq = content_id_seq[:-1].copy()
#         skill_seq += (answered_correctly_seq[:-1] == 1) * self.n_skill
        
#         return skill_seq,elapsed_time_seq,content_seq,label_seq,lag_time_seq

In [ ]:
# TEST_SIZE = 0.1

# train, val = train_test_split(group, test_size = TEST_SIZE)

In [ ]:
# train_dataset = UniDataset(train, n_content_id, max_seq=MAX_SEQ)
# train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
# del train

In [ ]:
# val_dataset = UniDataset(val, n_content_id, max_seq=MAX_SEQ)
# val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
# del val

In [ ]:
# sample_batch = next(iter(train_dataloader))
# sample_batch[0].shape, sample_batch[1].shape, sample_batch[2].shape, sample_batch[3].shape, sample_batch[4].shape

## Define model

In [ ]:
# def future_mask(seq_length):
#     future_mask = (np.triu(np.ones([seq_length, seq_length]), k = 1)).astype('bool')
#     return torch.from_numpy(future_mask)

# future_mask(5)

In [ ]:
# class UniModel(nn.Module):
#     def __init__(self,emb_dim,seq_len,num_content_id_item,num_label_item,num_skill_item,num_elapsed_time_item,num_lag_time_item):
#         super(UniModel,self).__init__() 
        
#         self.enc_seq_content_id_emb = nn.Embedding(num_content_id_item, emb_dim)
        
#         self.dec_seq_label_emb = nn.Embedding(num_label_item, emb_dim)
#         self.dec_seq_skill_emb = nn.Embedding(num_skill_item, emb_dim)
#         self.dec_seq_elapsed_time_emb = nn.Embedding(num_elapsed_time_item, emb_dim)
#         self.dec_seq_lag_time_emb = nn.Embedding(num_lag_time_item, emb_dim)

#         self.pos_emb = nn.Embedding(seq_len, emb_dim)
        
#         self.transformer_model = nn.Transformer(d_model=emb_dim, nhead=16, num_encoder_layers=1)
#         self.pred = nn.Linear(emb_dim, 1)

        
#     def forward(self, skill_seq, elapsed_time_seq, content_seq, label_seq, lag_time_seq):
            
#             start_token = torch.Tensor(label_seq.size(0),1).fill_(0)
#             dec_label_seq = torch.cat((start_token.to(device),label_seq[:,:-1].to(device)),-1).to(device)
            
#             dec_label_seq = self.dec_seq_label_emb(dec_label_seq.long().to(device))
#             dec_skill_seq = self.dec_seq_skill_emb(skill_seq.long().to(device))
#             dec_elapsed_time_seq = self.dec_seq_elapsed_time_emb(elapsed_time_seq.long().to(device))
#             dec_lag_time_seq = self.dec_seq_lag_time_emb(lag_time_seq.long().to(device))
#             dec_seq = dec_label_seq+dec_skill_seq+dec_elapsed_time_seq+dec_lag_time_seq
#             dec_seq = dec_seq.permute(1,0,2)
            
#             enc_seq = self.enc_seq_content_id_emb(content_seq.long()).to(device)
#             pos_id = torch.arange(enc_seq.size(1)).unsqueeze(0).long().to(device)
#             pos = self.pos_emb(pos_id.long())
#             enc_seq = enc_seq+pos
#             enc_seq = enc_seq.permute(1,0,2)
            
#             trans_mask = future_mask(enc_seq.size(0)).to(device)
#             out = self.transformer_model(enc_seq, dec_seq, src_mask=trans_mask, tgt_mask=trans_mask)
#             out = out.permute(1, 0, 2)
#             out = self.pred(out)
            
#             return out.squeeze(-1)

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# device

In [ ]:
# def create_model(emb_dim,seq_len,num_content_id_item,num_label_item,num_skill_item,num_elapsed_time_item,num_lag_time_item):
#     return UniModel(emb_dim,seq_len-1,num_content_id_item,num_label_item,num_skill_item,num_elapsed_time_item,num_lag_time_item).to(device)
# model = create_model(EMBED_SIZE,MAX_SEQ,num_content_id_item,num_label_item,num_skill_item,num_elapsed_time_item,num_lag_time_item)
# model

In [ ]:
# skill_seq,elapsed_time_seq,content_seq,label_seq,lag_time_seq


In [ ]:
# pd.Series(sample_batch[0].numpy().reshape(-1,)).nunique(),pd.Series(sample_batch[0].numpy().reshape(-1,)).describe()

In [ ]:
# pd.Series(sample_batch[1].numpy().reshape(-1,)).nunique(),pd.Series(sample_batch[1].numpy().reshape(-1,)).describe()

In [ ]:
# pd.Series(sample_batch[2].numpy().reshape(-1,)).nunique(),pd.Series(sample_batch[2].numpy().reshape(-1,)).describe()

In [ ]:
# pd.Series(sample_batch[3].numpy().reshape(-1,)).nunique(),pd.Series(sample_batch[3].numpy().reshape(-1,)).describe()

In [ ]:
# pd.Series(sample_batch[4].numpy().reshape(-1,)).nunique(),pd.Series(sample_batch[4].numpy().reshape(-1,)).describe()

In [ ]:
# model(sample_batch[0].to(device), sample_batch[1].to(device), sample_batch[2].to(device),sample_batch[3].to(device),sample_batch[4].to(device))[0]

## Training

In [ ]:
# def load_from_item(item):
#     skill_seq = item[0].to(device).long()
#     elapsed_time_seq = item[1].to(device).long()
#     content_seq = item[2].to(device).long()
#     label_seq = item[3].to(device).float()
#     lag_time_seq = item[4].to(device).long()
#     content_mask = (content_seq != 0)
#     return skill_seq, elapsed_time_seq, content_seq, label_seq, lag_time_seq, content_mask

# def update_stats(tbar, train_loss, loss, output, label, num_corrects, num_total, labels, outs):
#     train_loss.append(loss.item())
#     pred = (torch.sigmoid(output) >= 0.5).long()
#     num_corrects += (pred == label).sum().item()
#     num_total += len(label)
#     labels.extend(label.view(-1).data.cpu().numpy())
#     outs.extend(output.view(-1).data.cpu().numpy())
#     tbar.set_description('loss - {:.4f}'.format(loss))
#     return num_corrects, num_total

# def train_epoch(model, dataloader, optim, criterion, scheduler, device="cpu"):
#     model.train()
    
#     train_loss = []
#     num_corrects = 0
#     num_total = 0
#     labels = []
#     outs = []
    
#     tbar = tqdm(dataloader)
#     for item in tbar:
#         skill_seq, elapsed_time_seq, content_seq, label_seq, lag_time_seq, content_mask = load_from_item(item)
        
#         optim.zero_grad()
#         output = model(skill_seq, elapsed_time_seq, content_seq, label_seq, lag_time_seq)
        
        
#         output = torch.masked_select(output, content_mask)
#         label_seq = torch.masked_select(label_seq, content_mask)
        
#         loss = criterion(output, label_seq)
#         loss.backward()
#         optim.step()
#         scheduler.step()
        
#         tbar.set_description('loss - {:.4f}'.format(loss))

# def val_epoch(model, val_iterator, criterion, device="cpu"):
#     model.eval()

#     train_loss = []
#     num_corrects = 0
#     num_total = 0
#     labels = []
#     outs = []

#     tbar = tqdm(val_iterator)
#     for item in tbar:
#         skill_seq, elapsed_time_seq, content_seq, label_seq, lag_time_seq, content_mask = load_from_item(item)

#         with torch.no_grad():
#             output = model(skill_seq, elapsed_time_seq, content_seq, label_seq, lag_time_seq)
        
#         output = torch.masked_select(output, content_mask)
#         label_seq = torch.masked_select(label_seq, content_mask)

#         loss = criterion(output, label_seq)
        
#         num_corrects, num_total = update_stats(tbar, train_loss, loss, output, label_seq, num_corrects, num_total, labels, outs)
#     acc = num_corrects / num_total
#     auc = roc_auc_score(labels, outs)
#     loss = np.average(train_loss)

#     return loss, acc, auc


In [ ]:
# def do_train():
#     optimizer = torch.optim.Adam(model.parameters(), lr=LR)
#     criterion = nn.BCEWithLogitsLoss()
#     scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=LR, 
#                                                     steps_per_epoch=len(train_dataloader), epochs=EPOCHS)
#     model.to(device)
#     criterion.to(device)
#     best_auc = 0.0
#     for epoch in range(EPOCHS):
#         train_epoch(model, train_dataloader, optimizer, criterion, scheduler, device)
#         val_loss, avl_acc, val_auc = val_epoch(model, val_dataloader, criterion, device)
#         print(f"epoch - {epoch + 1} val_loss - {val_loss:.3f} val_acc - {avl_acc:.3f} val_auc - {val_auc:.3f}")
#         if best_auc < val_auc:
#             print(f'epoch - {epoch + 1} best model with val auc: {val_auc}')
#             best_auc = val_auc
#         torch.save(model.state_dict(), MODEL_PATH)

In [ ]:
# do_train()

In [ ]:
# LR = 2e-4
# EPOCHS = 3

# do_train()

# Predict (Notice! This predict code is not fix to adapt UniNet)

In [ ]:
# model = create_model()
# model.load_state_dict(torch.load(MODEL_PATH))
# model.to(device)

In [ ]:
# class TestDataset(Dataset):
#     def __init__(self, samples, test_df, n_skill, max_seq=100):
#         super(TestDataset, self).__init__()
#         self.samples, self.user_ids, self.test_df = samples, [x for x in test_df["user_id"].unique()], test_df
#         self.n_skill, self.max_seq = n_skill, max_seq

#     def __len__(self):
#         return self.test_df.shape[0]
    
#     def __getitem__(self, index):
#         test_info = self.test_df.iloc[index]
        
#         user_id = test_info['user_id']
#         target_id = test_info['content_id']
        
#         content_id_seq = np.zeros(self.max_seq, dtype=int)
#         answered_correctly_seq = np.zeros(self.max_seq, dtype=int)
        
#         if user_id in self.samples.index:
#             content_id, answered_correctly = self.samples[user_id]
            
#             seq_len = len(content_id)
            
#             if seq_len >= self.max_seq:
#                 content_id_seq = content_id[-self.max_seq:]
#                 answered_correctly_seq = answered_correctly[-self.max_seq:]
#             else:
#                 content_id_seq[-seq_len:] = content_id
#                 answered_correctly_seq[-seq_len:] = answered_correctly
                
#         x = content_id_seq[1:].copy()
#         x += (answered_correctly_seq[1:] == 1) * self.n_skill
        
#         questions = np.append(content_id_seq[2:], [target_id])
        
#         return x, questions

In [ ]:
# import riiideducation

# env = riiideducation.make_env()
# iter_test = env.iter_test()

In [ ]:
# import psutil

# model.eval()

# prev_test_df = None

# for (test_df, sample_prediction_df) in tqdm(iter_test):
    
#     if (prev_test_df is not None) & (psutil.virtual_memory().percent<90):
#         print(psutil.virtual_memory().percent)
#         prev_test_df['answered_correctly'] = eval(test_df['prior_group_answers_correct'].iloc[0])
#         prev_test_df = prev_test_df[prev_test_df.content_type_id == False]
#         prev_group = prev_test_df[['user_id', 'content_id', 'answered_correctly']].groupby('user_id').apply(lambda r: (
#             r['content_id'].values,
#             r['answered_correctly'].values))
#         for prev_user_id in prev_group.index:
#             prev_group_content = prev_group[prev_user_id][0]
#             prev_group_answered_correctly = prev_group[prev_user_id][1]
#             if prev_user_id in group.index:
#                 group[prev_user_id] = (np.append(group[prev_user_id][0], prev_group_content), 
#                                        np.append(group[prev_user_id][1], prev_group_answered_correctly))
#             else:
#                 group[prev_user_id] = (prev_group_content, prev_group_answered_correctly)
            
#             if len(group[prev_user_id][0]) > MAX_SEQ:
#                 new_group_content = group[prev_user_id][0][-MAX_SEQ:]
#                 new_group_answered_correctly = group[prev_user_id][1][-MAX_SEQ:]
#                 group[prev_user_id] = (new_group_content, new_group_answered_correctly)
                
#     prev_test_df = test_df.copy()
#     test_df = test_df[test_df.content_type_id == False]
    
#     test_dataset = TestDataset(group, test_df, n_skill, max_seq=MAX_SEQ)
#     test_dataloader = DataLoader(test_dataset, batch_size=len(test_df), shuffle=False)
    
#     item = next(iter(test_dataloader))
#     x = item[0].to(device).long()
#     target_id = item[1].to(device).long()
    
#     with torch.no_grad():
#         output, _ = model(x, target_id)
        
#     output = torch.sigmoid(output)
#     output = output[:, -1]
#     test_df['answered_correctly'] = output.cpu().numpy()
#     env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])

In [ ]:
# test_df

In [ ]:
# test_dataset = TestDataset(group, test_df, n_skill, max_seq=MAX_SEQ)

In [ ]:
# # Save to pickle to usage in other notebooks
# group.to_pickle('/kaggle/working/group.pkl')

In [ ]:
import riiideducation
env = riiideducation.make_env()
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    test_df['answered_correctly'] = 0.5
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])